In [48]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [49]:
from pyod.models.pca import PCA
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_excel('./ML_data.xlsx')
df.head()

,Type,AB,Site,Impurity,∆H (A-rich),∆H (B-rich),(+2/+1),(+1/0),(0/-1),(-1/-2),Unnamed: 10,Unnamed: 11,Descriptors ->,Z_A,Z_B,PBE_delta_H,PBE_latt_const,PBE_gap,Eps_elec,Eps_ion,CM1,CM2,CM3,CM4,CM5,CM6,CM7,CM8,Ion_rad,BP,MP,Density,At_wt,ICSD_vol,Cov_rad,Ion_Energy,At_rad_1,Elec_Aff,At_rad_2,At_vol,Mend_num,Ion_pot_1,Ion_pot_2,Thermal_expn,Sp_heat_cap,Therm_cond,Elec_cond,Heat_fusion,Heat_vap,Electronegativity,At_num,Period,Group,Valence,Ox_state
0,IV-IV,SiC,M_i_A,In,15.962579,15.962579,3.891344,4.660228,5.108776,5.628670,NaN,NaN,NaN,14,6,-0.42,4.25,1.27,18.06,2.27,373.01,373.01,373.01,373.01,138.44,138.44,138.44,79.93,0.80,2350.0,429.78,7.31,114.82,26.1,1.44,558.3,1.66,29.0,2.00,15.70,75,5.79,18.87,32.1,0.23,81.60,3.4,3.26,226.34,1.78,49,5,13,3,3
1,IV-IV,SiC,M_i_B,La,19.971847,19.554390,3.863351,4.538656,4.991232,5.555708,NaN,NaN,NaN,14,6,-0.42,4.25,1.27,18.06,2.27,375.78,375.78,375.78,216.96,185.96,185.96,185.96,185.96,1.16,3737.0,1191.00,6.15,138.91,37.4,1.25,538.1,1.88,50.0,2.74,20.73,13,5.58,11.06,5.2,0.19,13.50,1.9,11.30,399.57,1.10,57,6,3,3,3
2,IV-IV,SiC,M_i_neut,Zn,7.156721,7.156721,3.206206,4.244346,4.734064,5.363217,NaN,NaN,NaN,14,6,-0.42,4.25,1.27,18.06,2.27,152.25,238.53,238.53,238.53,102.23,102.23,102.23,65.25,0.74,1180.0,692.73,7.13,65.39,15.1,1.25,906.4,1.33,0.0,1.53,9.20,69,9.39,17.96,30.2,0.39,116.00,16.9,7.38,115.30,1.65,30,4,12,2,2
3,IV-IV,SiC,M_i_B,Al,6.950529,7.109779,3.502748,4.242487,4.649842,5.319256,NaN,NaN,NaN,14,6,-0.42,4.25,1.27,18.06,2.27,85.70,85.70,85.70,49.48,42.41,42.41,42.41,42.41,0.54,2740.0,933.50,2.70,26.98,16.6,1.18,577.6,1.43,45.0,1.62,10.00,73,5.99,18.83,23.1,0.90,237.00,37.7,10.70,290.80,1.61,13,3,13,3,3
4,IV-IV,SiC,M_B,I,8.783475,9.200933,3.614499,4.220773,5.057632,5.608287,NaN,NaN,NaN,14,6,-0.42,4.25,1.27,18.06,2.27,403.46,403.46,403.46,403.46,6874.36,105.89,105.89,105.89,2.20,457.5,386.70,4.93,126.90,42.5,1.33,1008.4,1.33,295.3,1.32,25.74,96,10.45,19.13,87.0,0.14,0.45,0.0,7.76,20.90,2.66,53,5,17,7,1


In [5]:
df.dropna(axis=1, inplace=True)
df.head()

,Type,AB,Site,Impurity,∆H (A-rich),∆H (B-rich),(+2/+1),(+1/0),(0/-1),(-1/-2),Z_A,Z_B,PBE_delta_H,PBE_latt_const,PBE_gap,Eps_elec,Eps_ion,CM1,CM2,CM3,CM4,CM5,CM6,CM7,CM8,Ion_rad,BP,MP,Density,At_wt,ICSD_vol,Cov_rad,Ion_Energy,At_rad_1,Elec_Aff,At_rad_2,At_vol,Mend_num,Ion_pot_1,Ion_pot_2,Thermal_expn,Sp_heat_cap,Therm_cond,Elec_cond,Heat_fusion,Heat_vap,Electronegativity,At_num,Period,Group,Valence,Ox_state
0,IV-IV,SiC,M_i_A,In,15.962579,15.962579,3.891344,4.660228,5.108776,5.628670,14,6,-0.42,4.25,1.27,18.06,2.27,373.01,373.01,373.01,373.01,138.44,138.44,138.44,79.93,0.80,2350.0,429.78,7.31,114.82,26.1,1.44,558.3,1.66,29.0,2.00,15.70,75,5.79,18.87,32.1,0.23,81.60,3.4,3.26,226.34,1.78,49,5,13,3,3
1,IV-IV,SiC,M_i_B,La,19.971847,19.554390,3.863351,4.538656,4.991232,5.555708,14,6,-0.42,4.25,1.27,18.06,2.27,375.78,375.78,375.78,216.96,185.96,185.96,185.96,185.96,1.16,3737.0,1191.00,6.15,138.91,37.4,1.25,538.1,1.88,50.0,2.74,20.73,13,5.58,11.06,5.2,0.19,13.50,1.9,11.30,399.57,1.10,57,6,3,3,3
2,IV-IV,SiC,M_i_neut,Zn,7.156721,7.156721,3.206206,4.244346,4.734064,5.363217,14,6,-0.42,4.25,1.27,18.06,2.27,152.25,238.53,238.53,238.53,102.23,102.23,102.23,65.25,0.74,1180.0,692.73,7.13,65.39,15.1,1.25,906.4,1.33,0.0,1.53,9.20,69,9.39,17.96,30.2,0.39,116.00,16.9,7.38,115.30,1.65,30,4,12,2,2
3,IV-IV,SiC,M_i_B,Al,6.950529,7.109779,3.502748,4.242487,4.649842,5.319256,14,6,-0.42,4.25,1.27,18.06,2.27,85.70,85.70,85.70,49.48,42.41,42.41,42.41,42.41,0.54,2740.0,933.50,2.70,26.98,16.6,1.18,577.6,1.43,45.0,1.62,10.00,73,5.99,18.83,23.1,0.90,237.00,37.7,10.70,290.80,1.61,13,3,13,3,3
4,IV-IV,SiC,M_B,I,8.783475,9.200933,3.614499,4.220773,5.057632,5.608287,14,6,-0.42,4.25,1.27,18.06,2.27,403.46,403.46,403.46,403.46,6874.36,105.89,105.89,105.89,2.20,457.5,386.70,4.93,126.90,42.5,1.33,1008.4,1.33,295.3,1.32,25.74,96,10.45,19.13,87.0,0.14,0.45,0.0,7.76,20.90,2.66,53,5,17,7,1


In [6]:
descriptors = df.columns[10:]
descriptors

Index(['Z_A', 'Z_B', 'PBE_delta_H', 'PBE_latt_const', 'PBE_gap', 'Eps_elec',
       'Eps_ion', 'CM1', 'CM2', 'CM3', 'CM4', 'CM5', 'CM6', 'CM7', 'CM8',
       'Ion_rad', 'BP', 'MP', 'Density', 'At_wt', 'ICSD_vol', 'Cov_rad',
       'Ion_Energy', 'At_rad_1', 'Elec_Aff', 'At_rad_2', 'At_vol', 'Mend_num',
       'Ion_pot_1', 'Ion_pot_2', 'Thermal_expn', 'Sp_heat_cap', 'Therm_cond',
       'Elec_cond', 'Heat_fusion', 'Heat_vap', 'Electronegativity', 'At_num',
       'Period', 'Group', 'Valence', 'Ox_state'],
      dtype='object')

In [7]:
output = df.columns[4:10]
output

Index(['∆H (A-rich)', '∆H (B-rich)', '(+2/+1)', '(+1/0)', '(0/-1)', '(-1/-2)'], dtype='object')

In [9]:
def counter(dataframe, column):
    
    count = pd.value_counts(dataframe[column])
    count_df = count.to_frame()
    count_df['percent']=round(count_df[column]/sum(count)*100,2)
    
    print('Total entries: ', sum(count))
    
    return count_df

## PCA using ``pyOD``

- what DFT data is anomalous? Removing 10% of the data that appears anomalous based on the "target" values calculated from DFT, or the "descriptor" values.

In [58]:
def myPCA(dataframe, data_type):
    array = np.array(dataframe[data_type])
    
    # using PCA model from pymod
    # model will identify ~ 10% of data as outliers
    clf = PCA(contamination=0.1)

    # fitting the data
    clf.fit(array)

    # classifying the targets as either inliers(0) or outliers(1)
    ypred = clf.predict(array)
    
    # df of outliers
    df_outlier = dataframe[ypred == 1]

    # df without outliers
    df_nooutlier = dataframe[ypred == 0]
    
    return df_outlier, df_nooutlier

### PCA with target vals

In [59]:
df_pca_out_tar, df_pca_noout_tar = myPCA(df, output)

### PCA with descriptor vals

In [60]:
df_pca_out_des, df_pca_noout_des = myPCA(df, descriptors)

### Similar outlier rows between the PCA models

In [61]:
df_comm = pd.concat([df_pca_out_tar, df_pca_out_des])

df_comm = df_comm.reset_index(drop=True)

df_gpby = df_comm.groupby(list(df_comm.columns))

idx = [x[0] for x in df_gpby.groups.values() if len(x) != 1]
df_comm.reindex(idx)

,Type,AB,Site,Impurity,∆H (A-rich),∆H (B-rich),(+2/+1),(+1/0),(0/-1),(-1/-2),Z_A,Z_B,PBE_delta_H,PBE_latt_const,PBE_gap,Eps_elec,Eps_ion,CM1,CM2,CM3,CM4,CM5,CM6,CM7,CM8,Ion_rad,BP,MP,Density,At_wt,ICSD_vol,Cov_rad,Ion_Energy,At_rad_1,Elec_Aff,At_rad_2,At_vol,Mend_num,Ion_pot_1,Ion_pot_2,Thermal_expn,Sp_heat_cap,Therm_cond,Elec_cond,Heat_fusion,Heat_vap,Electronegativity,At_num,Period,Group,Valence,Ox_state
45,II-VI,CdO,M_B,Re,10.231465,18.380317,0.570528,1.135005,1.624657,2.211828,48,8,-2.04,5.15,0.47,9.63,6.00,1613.57,1613.57,1613.57,1613.57,15816.74,164.68,164.68,164.68,0.53,5870.00,3455.00,21.00,186.21,14.7,1.28,760.0,1.37,14.00,1.97,8.85,54,7.88,12.60,6.2,0.14,47.90,5.8,33.05,707.10,1.90,75,6,7,7,7
8,III-V,AlN,M_i_B,Ca,12.449316,14.825316,3.086236,4.097375,4.988731,5.717568,13,7,-8.00,4.40,3.28,6.80,3.15,118.29,118.29,118.29,68.30,73.55,73.55,73.55,73.55,1.00,1757.00,1112.00,1.55,40.08,44.1,1.74,589.8,1.97,-156.00,2.23,25.90,7,6.11,11.87,22.3,0.65,200.00,31.3,8.53,154.67,1.00,20,4,2,2,2
22,III-V,AlN,M_i_neut,Li,2.692778,4.000778,-0.650621,3.653570,4.667719,5.550833,13,7,-8.00,4.40,3.28,6.80,3.15,13.66,21.40,21.40,21.40,11.52,11.52,11.52,7.35,0.76,1615.00,453.70,0.53,6.94,21.6,1.23,520.2,1.55,59.80,2.05,13.10,1,5.39,76.64,46.0,3.58,84.70,11.7,3.00,147.10,0.98,3,2,1,1,1
34,III-V,BAs,M_i_neut,Cs,18.241325,18.352521,1.178237,2.602052,3.281467,3.712016,5,33,-0.07,4.81,1.22,25.40,0.00,88.06,137.97,137.97,137.97,910.60,910.60,910.60,581.22,1.74,944.00,301.54,1.87,132.91,67.5,2.35,375.7,2.67,45.49,3.34,71.07,5,3.89,25.10,100.0,0.24,35.90,5.3,2.09,67.74,0.79,55,6,1,1,1
15,III-V,BN,M_A,As,14.243859,5.391471,2.977147,3.900389,5.197039,6.645629,5,7,-7.65,3.63,4.45,5.83,0.00,2204.97,64.34,64.34,64.34,147.10,147.10,147.10,147.10,0.58,876.00,1090.00,5.78,74.92,21.5,1.20,947.0,1.39,77.00,1.33,13.10,84,9.81,18.63,15.4,0.33,50.00,3.8,27.70,32.40,2.16,33,4,15,5,3
6,III-V,BN,M_A,Pb,17.053058,9.404671,-0.954139,4.167417,5.412812,6.754755,5,7,-7.65,3.63,4.45,5.83,0.00,19594.01,159.88,159.88,159.88,365.53,365.53,365.53,365.53,1.19,2023.00,600.65,11.35,207.20,30.3,1.47,715.5,1.75,110.00,1.81,18.17,81,7.42,15.03,28.8,0.13,35.30,4.8,4.77,177.90,2.33,82,6,14,4,2
21,III-V,BN,M_B,F,7.784592,12.883516,2.340939,3.682828,5.552112,6.721851,5,7,-7.65,3.63,4.45,5.83,0.00,28.66,28.66,28.66,28.66,97.53,24.57,24.57,24.57,1.33,85.00,53.55,1.70,19.00,16.4,0.72,1681.0,0.72,328.00,0.57,12.60,93,17.42,34.97,1800.0,0.82,0.03,12.0,0.26,3.27,3.98,9,2,17,1,1
31,III-V,BN,M_B,Li,10.045186,18.413573,2.096747,2.926480,3.270529,3.270529,5,7,-7.65,3.63,4.45,5.83,0.00,9.55,9.55,9.55,9.55,6.98,8.19,8.19,8.19,0.76,1615.00,453.70,0.53,6.94,21.6,1.23,520.2,1.55,59.80,2.05,13.10,1,5.39,76.64,46.0,3.58,84.70,11.7,3.00,147.10,0.98,3,2,1,1,1
26,III-V,BN,M_i_B,Cl,20.895758,19.059758,-0.399877,3.357321,4.462068,6.774461,5,7,-7.65,3.63,4.45,5.83,0.00,46.88,46.88,46.88,27.06,75.78,75.78,75.78,75.78,1.81,239.18,172.17,3.21,35.45,29.0,0.99,1251.1,0.99,348.80,0.97,16.90,94,12.97,23.81,0.0,0.48,0.01,0.0,3.21,10.20,3.16,17,3,17,7,1
33,IV-IV,C,M_A,Br,18.300591,18.304605,1.044903,2.636744,3.931709,5.080552,6,6,0.00,3.57,4.12,9.49,0.46,2539.41,83.10,83.10,83.10,135.71,135.71,135.71,135.71,1.96,331.85,265.95,3.12,79.90,65.1,1.14,1139.9,1.14,324.60,1.12,25.60,95,11.81,21.80,0.0,0.23,0.12,0.0,5.29,14.72,2.96,35,4,17,7,1


In [62]:
counter(df_pca_out_tar, 'Type')

Total entries:  86


,Type,percent
IV-IV,63,73.26
III-V,16,18.60
II-VI,7,8.14


In [63]:
counter(df_pca_noout_tar, 'Type')

Total entries:  767


,Type,percent
II-VI,499,65.06
III-V,139,18.12
IV-IV,129,16.82


In [64]:
counter(df_pca_out_des, 'Type')

Total entries:  86


,Type,percent
II-VI,45,52.33
III-V,22,25.58
IV-IV,19,22.09


In [65]:
counter(df_pca_noout_des, 'Type')

Total entries:  767


,Type,percent
II-VI,461,60.10
IV-IV,173,22.56
III-V,133,17.34


In [66]:
df_nooutliers = df[(df['∆H (A-rich)'] <= 10) & (df['∆H (B-rich)'] <= 10)]
df_nooutliers

,Type,AB,Site,Impurity,∆H (A-rich),∆H (B-rich),(+2/+1),(+1/0),(0/-1),(-1/-2),Z_A,Z_B,PBE_delta_H,PBE_latt_const,PBE_gap,Eps_elec,Eps_ion,CM1,CM2,CM3,CM4,CM5,CM6,CM7,CM8,Ion_rad,BP,MP,Density,At_wt,ICSD_vol,Cov_rad,Ion_Energy,At_rad_1,Elec_Aff,At_rad_2,At_vol,Mend_num,Ion_pot_1,Ion_pot_2,Thermal_expn,Sp_heat_cap,Therm_cond,Elec_cond,Heat_fusion,Heat_vap,Electronegativity,At_num,Period,Group,Valence,Ox_state
2,IV-IV,SiC,M_i_neut,Zn,7.156721,7.156721,3.206206,4.244346,4.734064,5.363217,14,6,-0.42,4.25,1.27,18.06,2.27,152.25,238.53,238.53,238.53,102.23,102.23,102.23,65.25,0.74,1180.0,692.73,7.13,65.39,15.1,1.25,906.4,1.33,0.0,1.53,9.20,69,9.39,17.96,30.2,0.39,116.00,16.9,7.38,115.30,1.65,30,4,12,2,2
3,IV-IV,SiC,M_i_B,Al,6.950529,7.109779,3.502748,4.242487,4.649842,5.319256,14,6,-0.42,4.25,1.27,18.06,2.27,85.70,85.70,85.70,49.48,42.41,42.41,42.41,42.41,0.54,2740.0,933.50,2.70,26.98,16.6,1.18,577.6,1.43,45.0,1.62,10.00,73,5.99,18.83,23.1,0.90,237.00,37.7,10.70,290.80,1.61,13,3,13,3,3
4,IV-IV,SiC,M_B,I,8.783475,9.200933,3.614499,4.220773,5.057632,5.608287,14,6,-0.42,4.25,1.27,18.06,2.27,403.46,403.46,403.46,403.46,6874.36,105.89,105.89,105.89,2.20,457.5,386.70,4.93,126.90,42.5,1.33,1008.4,1.33,295.3,1.32,25.74,96,10.45,19.13,87.0,0.14,0.45,0.0,7.76,20.90,2.66,53,5,17,7,1
5,IV-IV,SiC,M_i_A,Ru,9.266514,8.849056,3.336793,4.219286,4.660660,5.267393,14,6,-0.42,4.25,1.27,18.06,2.27,334.95,334.95,334.95,334.95,124.32,124.32,124.32,71.77,0.62,4425.0,2610.00,12.37,101.07,14.0,1.25,711.0,1.32,110.0,1.89,8.30,56,7.37,16.76,6.4,0.24,117.00,14.9,25.52,567.77,2.20,44,5,8,8,3
7,IV-IV,SiC,M_B,Rh,5.273336,5.273336,1.660711,4.106883,4.695697,5.273310,14,6,-0.42,4.25,1.27,18.06,2.27,342.56,342.56,342.56,342.56,4641.73,89.90,89.90,89.90,0.67,3970.0,2236.00,12.41,102.91,11.5,1.25,720.0,1.34,120.0,1.83,8.30,59,7.46,18.08,8.2,0.24,150.00,23.0,21.76,495.39,2.28,45,5,9,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,IV-IV,SnC,M_A,Cr,-1.598407,0.946053,-2.750073,-2.404091,-1.048124,-0.520923,50,6,1.51,5.09,0.62,10.71,5.61,1026.78,333.57,333.57,333.57,65.37,65.37,65.37,65.37,0.62,2945.0,2130.00,7.19,52.00,12.0,1.18,652.8,1.25,64.0,1.85,7.23,49,6.77,16.50,4.9,0.45,93.70,7.9,20.00,339.50,1.66,24,4,6,6,3
849,IV-IV,SnC,M_A,Pb,1.461677,7.276597,-2.901850,-2.440133,-0.729618,-0.075188,50,6,1.51,5.09,0.62,10.71,5.61,19594.01,1139.70,1139.70,1139.70,223.33,223.33,223.33,223.33,1.19,2023.0,600.65,11.35,207.20,30.3,1.47,715.5,1.75,110.0,1.81,18.17,81,7.42,15.03,28.8,0.13,35.30,4.8,4.77,177.90,2.33,82,6,14,4,2
850,III-V,GaN,M_B,P,1.920718,8.669929,-3.098571,-2.446655,0.473968,1.434063,31,7,-6.13,4.55,1.55,15.00,0.26,236.11,236.11,236.11,236.11,332.34,32.65,32.65,32.65,0.17,553.0,317.30,1.82,30.97,18.9,1.06,1011.7,1.28,71.7,1.23,17.00,83,10.49,19.73,127.0,0.77,0.23,0.0,0.63,12.40,2.19,15,3,15,5,5
851,III-V,GaN,M_A,Tl,3.862552,-2.269993,-3.297060,-2.634126,0.266647,1.270077,31,7,-6.13,4.55,1.55,15.00,0.26,19025.41,780.76,780.76,780.76,287.90,287.90,287.90,287.90,1.59,1746.0,577.00,11.85,204.38,28.5,1.48,589.3,1.71,30.0,2.08,17.20,76,6.11,20.43,29.9,0.13,46.10,5.6,4.27,162.09,2.04,81,6,13,3,1


In [69]:
df1, dfnout_pca_tar = myPCA(df_nooutliers, output)

In [75]:
counter(df1, 'Type')

Total entries:  77


,Type,percent
IV-IV,63,81.82
III-V,9,11.69
II-VI,5,6.49


In [70]:
df3, dfnout_pca_des = myPCA(df_nooutliers, descriptors)

In [76]:
counter(df3, 'Type')

Total entries:  77


,Type,percent
II-VI,41,53.25
III-V,20,25.97
IV-IV,16,20.78


---------------------
---------

## Evaluating RFR on data after PCA

In [39]:
def RFR_abbr(df, o_start=4, o_end=10):
    '''
    o_start: int. column index of target value. (4 is the beginning)
    o_end: int. column index of target value. (10 is the end)
    '''
    descriptors = df.columns[10:]
    output = df.columns[o_start:o_end]
    train,test = train_test_split(df,test_size=0.22, random_state=130)
    clf = RandomForestRegressor(n_jobs=2, random_state=130)
    
    frames_list = []
    train_rmse_list = [] 
    test_rmse_list = []
    
    for o in output:
        clf.fit(train[descriptors], train[o])

        trainpred = clf.predict(train[descriptors])
        testpred = clf.predict(test[descriptors])
        
        train_rmse = mean_squared_error(train[o],trainpred, squared=False)
        test_rmse = mean_squared_error(test[o],testpred, squared=False)
        
        train_rmse_list.append(train_rmse)
        test_rmse_list.append(test_rmse)

    
    d = {'output': output, 'train rmse': train_rmse_list, 'test rmse': test_rmse_list}
    
    rmse_df = pd.DataFrame(data=d)

    return rmse_df

In [40]:
def RFR_type(df, o_start=4, o_end=10):
    
    descriptors = df.columns[10:]
    output = df.columns[o_start:o_end]
    train,test = train_test_split(df,test_size=0.22, random_state=130)
    clf = RandomForestRegressor(n_jobs=2, random_state=130)
    
    train_26 = train[train['Type']=='II-VI']
    train_35 = train[train['Type']=='III-V']
    train_44 =  train[train['Type']=='IV-IV']

    test_26 = test[test['Type']=='II-VI']
    test_35 = test[test['Type']=='III-V']
    test_44 = test[test['Type']=='IV-IV']
    
    traintest_list = [(train_26, test_26),(train_35, test_35),(train_44, test_44)]
    
    tt_dict = {}
    
    for tt in traintest_list:
        
        key = str(tt[0].Type.unique())
        
        train_rmse_list = [] 
        test_rmse_list = []
        
        for o in output:
            clf.fit(train[descriptors], train[o])

            trainpred = clf.predict(tt[0][descriptors])
            testpred = clf.predict(tt[1][descriptors])

            train_rmse = mean_squared_error(tt[0][o],trainpred, squared=False)
            test_rmse = mean_squared_error(tt[1][o],testpred, squared=False)

            
            train_rmse_list.append(train_rmse)
            test_rmse_list.append(test_rmse)
        
        #print('train', train_rmse_list)
        #print('test', test_rmse_list)
            
        d = {'output': output, 'train rmse': train_rmse_list, 'test rmse': test_rmse_list}

        rmse_df = pd.DataFrame(data=d)
        
        tt_dict[key] = (rmse_df)
    
    return tt_dict

### PCA target

In [41]:
RFR_abbr(df_pca_noout_tar)

,output,train rmse,test rmse
0,∆H (A-rich),0.521188,1.711316
1,∆H (B-rich),0.668668,1.969930
2,(+2/+1),0.180982,0.627352
3,(+1/0),0.187063,0.550364
4,(0/-1),0.170332,0.490597
5,(-1/-2),0.127100,0.344000


In [42]:
RFR_type(df_pca_noout_tar)

{"['II-VI']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.398731   1.165730
 1  ∆H (B-rich)    0.532737   1.600196
 2      (+2/+1)    0.170987   0.421911
 3       (+1/0)    0.174467   0.430523
 4       (0/-1)    0.159486   0.408324
 5      (-1/-2)    0.122719   0.319528,
 "['III-V']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.742813   1.975805
 1  ∆H (B-rich)    0.960489   2.639330
 2      (+2/+1)    0.193675   0.749402
 3       (+1/0)    0.193055   0.584927
 4       (0/-1)    0.201011   0.646550
 5      (-1/-2)    0.119188   0.393656,
 "['IV-IV']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.645091   2.680194
 1  ∆H (B-rich)    0.761205   2.294364
 2      (+2/+1)    0.204237   0.972028
 3       (+1/0)    0.225625   0.808873
 4       (0/-1)    0.175561   0.560239
 5      (-1/-2)    0.150858   0.369650}

### PCA descriptors

In [43]:
RFR_abbr(df_pca_noout_des)

,output,train rmse,test rmse
0,∆H (A-rich),0.690964,2.052092
1,∆H (B-rich),0.804967,2.739747
2,(+2/+1),0.215243,0.580923
3,(+1/0),0.204297,0.532751
4,(0/-1),0.163685,0.419417
5,(-1/-2),0.118267,0.340651


In [44]:
RFR_type(df_pca_noout_des)

{"['II-VI']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.445371   1.025345
 1  ∆H (B-rich)    0.650307   1.494498
 2      (+2/+1)    0.181521   0.390340
 3       (+1/0)    0.174682   0.446580
 4       (0/-1)    0.151340   0.380506
 5      (-1/-2)    0.110798   0.324668,
 "['III-V']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.840807   2.217076
 1  ∆H (B-rich)    0.983819   2.706248
 2      (+2/+1)    0.251696   0.622941
 3       (+1/0)    0.288901   0.491263
 4       (0/-1)    0.208790   0.451170
 5      (-1/-2)    0.125104   0.274852,
 "['IV-IV']":         output  train rmse  test rmse
 0  ∆H (A-rich)    1.057928   3.114466
 1  ∆H (B-rich)    1.028863   4.197471
 2      (+2/+1)    0.268930   0.807760
 3       (+1/0)    0.204346   0.688082
 4       (0/-1)    0.157925   0.464156
 5      (-1/-2)    0.133105   0.405730}

### df_nooutliers (formation energy values > 10 eV removed)

In [71]:
RFR_abbr(dfnout_pca_tar)

,output,train rmse,test rmse
0,∆H (A-rich),0.453549,1.239188
1,∆H (B-rich),0.524834,1.490647
2,(+2/+1),0.177215,0.509280
3,(+1/0),0.201798,0.488115
4,(0/-1),0.164993,0.451035
5,(-1/-2),0.119997,0.329110


In [72]:
RFR_type(dfnout_pca_tar)

{"['II-VI']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.351041   0.937208
 1  ∆H (B-rich)    0.439929   1.313003
 2      (+2/+1)    0.166860   0.438792
 3       (+1/0)    0.178266   0.407402
 4       (0/-1)    0.152526   0.463214
 5      (-1/-2)    0.116928   0.332916,
 "['III-V']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.612832   2.080744
 1  ∆H (B-rich)    0.739946   2.172493
 2      (+2/+1)    0.198038   0.711526
 3       (+1/0)    0.212959   0.690511
 4       (0/-1)    0.186090   0.443661
 5      (-1/-2)    0.127007   0.298336,
 "['IV-IV']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.659473   1.071020
 1  ∆H (B-rich)    0.596540   1.231551
 2      (+2/+1)    0.201566   0.509922
 3       (+1/0)    0.291917   0.519368
 4       (0/-1)    0.198201   0.407455
 5      (-1/-2)    0.126832   0.345094}

In [73]:
RFR_abbr(dfnout_pca_des)

,output,train rmse,test rmse
0,∆H (A-rich),0.483860,1.360507
1,∆H (B-rich),0.543609,1.492331
2,(+2/+1),0.197292,0.713207
3,(+1/0),0.194802,0.487206
4,(0/-1),0.167198,0.455665
5,(-1/-2),0.117659,0.322446


In [74]:
RFR_type(dfnout_pca_des)

{"['II-VI']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.377965   1.107437
 1  ∆H (B-rich)    0.450525   1.388783
 2      (+2/+1)    0.169078   0.509091
 3       (+1/0)    0.174737   0.486724
 4       (0/-1)    0.160910   0.467613
 5      (-1/-2)    0.115325   0.332616,
 "['III-V']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.684610   1.770361
 1  ∆H (B-rich)    0.798284   1.812151
 2      (+2/+1)    0.260531   0.443706
 3       (+1/0)    0.237212   0.453829
 4       (0/-1)    0.193921   0.449996
 5      (-1/-2)    0.129856   0.312493,
 "['IV-IV']":         output  train rmse  test rmse
 0  ∆H (A-rich)    0.594868   1.582138
 1  ∆H (B-rich)    0.574017   1.496213
 2      (+2/+1)    0.224681   1.151608
 3       (+1/0)    0.219765   0.509642
 4       (0/-1)    0.164330   0.429263
 5      (-1/-2)    0.114858   0.303376}

## PCA

In [ ]:
x = df.loc[:, descriptors].values
y = df.loc[:, output].values

x = StandardScaler().fit_transform(x)

In [ ]:
df_scaled = pd.DataFrame(data=x, columns=descriptors)
df_scaled.head()

In [ ]:
pca = PCA(n_components=2)

principal_comp = pca.fit_transform(x)

df_principal = pd.DataFrame(data=principal_comp, columns = ['comp1', 'comp2'])

df_principal.head()

In [ ]:
df_final = pd.concat([df_principal, df.Type], axis=1)
df_final.head()

In [ ]:
df_final.iloc[:,2]

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = list(types)
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = df_final['Type'] == target
    ax.scatter(df_final.loc[indicesToKeep, 'comp1']
               , df_final.loc[indicesToKeep, 'comp2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
df_final = pd.concat([df_principal, df.AB], axis=1)
df_final.head()

In [ ]:
import matplotlib.cm as cm

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = list(AB)
colors = cm.rainbow(np.linspace(0,1,len(AB)))
for target, color in zip(targets,colors):
    indicesToKeep = df_final['AB'] == target
    ax.scatter(df_final.loc[indicesToKeep, 'comp1']
               ,df_final.loc[indicesToKeep, 'comp2']
               , c = np.array([color]), s = 50)
ax.legend(targets)
ax.grid()
plt.show()